# Data Indexing and Selection

Hemos visto en detalle los métodos y herramientas para acceder, establecer y modificar valores en matrices NumPy.
Estos incluyen indexación (por ejemplo, ``arr[2, 1]``), corte (por ejemplo, ``arr[:, 1:5]``), enmascaramiento (por ejemplo, ``arr[arr > 0]``), indexación de fantasía (por ejemplo, ``arr[0, [1, 5]]``), y combinaciones de los mismos (por ejemplo, ``arr[:, [1, 5]]``).
**Aquí veremos formas similares de acceder y modificar valores en objetos Pandas ``Series`` y ``DataFrame``.
Si has usado los patrones de NumPy, los patrones correspondientes en Pandas te resultarán muy familiares, aunque hay algunas peculiaridades que debes tener en cuenta.

Empezaremos con el caso simple del objeto unidimensional ``Series``, y luego pasaremos al más complicado objeto bidimensional ``DataFrame``.

## Selección de datos en serie

Como vimos en la sección anterior, un objeto **``Series`` actúa en muchos aspectos como un array unidimensional de NumPy, y en muchos aspectos como un diccionario estándar de Python.**
Si mantenemos en mente estas dos analogías superpuestas, nos ayudará a entender los patrones de indexación y selección de datos en estos arrays.

### Series como diccionario

Al igual que un diccionario, el objeto ``Series`` proporciona una correspondencia entre una colección de claves y una colección de valores:

In [80]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [81]:
data['b']

0.5

También podemos utilizar expresiones y métodos de Python tipo diccionario para examinar las claves/índices y los valores:

In [82]:
'b' in data

True

In [83]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [84]:
data.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [85]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Los objetos ``Series`` pueden incluso modificarse con una sintaxis similar a la de un diccionario.
Del mismo modo que se puede ampliar un diccionario asignando una nueva clave, se puede ampliar una ``Serie`` asignando un nuevo valor de índice:

In [86]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas está tomando decisiones sobre la disposición de la memoria y la copia de datos que pueda ser necesario realizar; el usuario generalmente no necesita preocuparse por estos temas.

### Series como array unidimensional

Una ``Serie`` se basa en esta interfaz tipo diccionario y proporciona una selección de elementos tipo array a través de los mismos mecanismos básicos que los arrays de NumPy, es decir, *rebanadas*, *enmascaramiento* e *índice de fantasía*.
Algunos ejemplos son los siguientes:

In [87]:
# corte por índice explícito
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [88]:
# corte por índice entero implícito
data[0:2]

a    0.25
b    0.50
dtype: float64

In [89]:
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

In [90]:
# enmascaramiento
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [91]:
# indexación sofisticada
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

Entre ellos, el troceado puede ser la fuente de mayor confusión.

**Tenga en cuenta que cuando se corta con un índice explícito (es decir, ``data['a':'c']``), el índice final está *incluido* en la rebanada, mientras que cuando se corta con un índice implícito (es decir, ``data[0:2]``), el índice final está *excluido* de la rebanada.**

### Indexadores: loc, iloc, e ix

Estas convenciones de corte e indexación pueden ser fuente de confusión.
Por ejemplo, si tu ``Serie`` tiene un índice entero explícito, una operación de indexación como **`datos[1]`` usará los índices explícitos, mientras que una operación de corte como ``datos[1:3]`` usará el índice implícito al estilo Python.**

In [92]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [93]:
# explicit index when indexing
data[1]

'a'

In [94]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

Debido a esta confusión potencial en el caso de índices enteros, Pandas proporciona algunos atributos *indexer* especiales que exponen explícitamente ciertos esquemas de indexación.
No se trata de métodos funcionales, sino de atributos que exponen una interfaz de corte particular para los datos de la ``Serie``.

En primer lugar, el atributo **`loc`` permite la indexación y el corte que siempre hace referencia al índice explícito:**

In [95]:
data

1    a
3    b
5    c
dtype: object

In [96]:
data.loc[1]

'a'

In [97]:
data.loc[1:3]

1    a
3    b
dtype: object

El atributo ``iloc`` permite indexar y rebanar que siempre hace referencia al índice implícito estilo Python:

In [98]:
data

1    a
3    b
5    c
dtype: object

In [99]:
data.iloc[1]

'b'

In [100]:
data.iloc[1:2]

3    b
dtype: object

Un tercer atributo de indexación, ``ix``, es un híbrido de los dos, y para los objetos ``Series`` es equivalente a la indexación estándar basada en ``[]``.
El propósito del indexador ``ix`` se hará más evidente en el contexto de los objetos ``DataFrame``, que discutiremos en un momento.

Un principio rector del código Python es que "lo explícito es mejor que lo implícito".
La naturaleza explícita de ``loc`` y ``iloc`` los hace muy útiles para mantener un código limpio y legible; especialmente en el caso de índices de enteros, **recomiendo usarlos tanto para hacer el código más fácil de leer y entender, como para prevenir bugs sutiles debidos a la convención mixta de indexación/rebanado.**

## Selección de datos en DataFrame

Recordemos que un ``DataFrame`` actúa en muchos aspectos como una matriz bidimensional o estructurada, y en otros como un diccionario de estructuras ``Series`` que comparten el mismo índice.
Estas analogías pueden ser útiles para tener en cuenta a medida que exploramos la selección de datos dentro de esta estructura.

### DataFrame como diccionario

La primera analogía que vamos a considerar es el ``DataFrame`` como diccionario de objetos ``Series`` relacionados.
Volvamos a nuestro ejemplo de las áreas y poblaciones de los estados:

In [109]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
# cities = pd.Series({'California': "Sacramento", 'Texas': "Otra ciudad 1",
#                  'New York': "Albany", 'Florida': "Miami",
#                  'Illinois': "Otra ciudad 2"})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [102]:
# data.reset_index()

Se puede acceder a las ``Series`` individuales que componen las columnas del ``DataFrame`` mediante la indexación de tipo diccionario del nombre de la columna:

In [103]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

De forma equivalente, podemos utilizar el acceso de tipo atributo con nombres de columna que sean cadenas:

In [104]:
data.population

AttributeError: 'DataFrame' object has no attribute 'population'

In [ ]:
data.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Este acceso de columna de tipo atributo accede en realidad exactamente al mismo objeto que el acceso de tipo diccionario:

In [ ]:
data.area is data['area']

True

Aunque se trata de una abreviatura útil, hay que tener en cuenta que no funciona en todos los casos.
Por ejemplo, **si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con métodos del ``DataFrame``, este acceso tipo atributo no es posible.**
Por ejemplo, el ``DataFrame`` tiene un método ``pop()``, por lo que ``data.pop`` apuntará a éste en lugar de a la columna ``"pop"``:

In [ ]:
data.pop is data['pop']

False

En particular, debe evitar la tentación de intentar asignar columnas mediante atributos (es decir, utilice ``datos['pop'] = z`` en lugar de ``datos.pop = z``).

Al igual que con los objetos ``Series`` comentados anteriormente, esta sintaxis de tipo diccionario también se puede utilizar para modificar el objeto, en este caso añadiendo una nueva columna:

In [ ]:
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [110]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Esto muestra una vista previa de la sintaxis directa de la aritmética elemento a elemento entre objetos ``Series``; profundizaremos más en esto en [Operando con datos en Pandas](3_Operaciones-en-Pandas.ipynb).

### DataFrame como matriz bidimensional

Como se mencionó anteriormente, también podemos ver el ``DataFrame`` como una matriz bidimensional mejorada.
Podemos examinar la matriz de datos subyacente utilizando el atributo ``values``:

In [ ]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

Con esta imagen en mente, se pueden hacer muchas observaciones familiares sobre el propio ``DataFrame``.
**Por ejemplo, podemos transponer el ``DataFrame`` completo para intercambiar filas y columnas:**.

In [ ]:
data.T

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
pop,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


Cuando se trata de la indexación de objetos ``DataFrame``, sin embargo, está claro que el estilo de indexación de diccionario de las columnas impide nuestra **capacidad de simplemente tratarlo como una matriz NumPy.**
En particular, al pasar un único índice a un array se accede a una fila:

In [ ]:
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [ ]:
data.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

and passing a single "index" to a ``DataFrame`` accesses a column:

In [ ]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Por lo tanto, para la indexación estilo array, necesitamos otra convención.
Aquí Pandas utiliza de nuevo los indexadores ``loc``, ``iloc``, y ``ix`` mencionados anteriormente.
Usando el indexador ``iloc``, podemos indexar el array subyacente como si fuera un simple array NumPy (usando el índice implícito estilo Python), pero el índice ``DataFrame`` y las etiquetas de las columnas se mantienen en el resultado:

In [108]:
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [ ]:
data.iloc[:4, 1:3]

,pop,density
California,38332521,90.413926
Texas,26448193,38.018740
New York,19651127,139.076746
Florida,19552860,114.806121


Del mismo modo, utilizando el indexador ``loc`` podemos indexar los datos subyacentes en un estilo similar a un array pero utilizando el índice explícito y los nombres de las columnas:

In [107]:
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [ ]:
data.loc['Texas':'New York', 'pop':'density']

""
Texas
New York


**IX ESTÁ OBSOLETO**

El indexador ``ix`` permite un híbrido de estos dos enfoques:

In [ ]:


#data.ix[:3, :'pop']

NameError: name 'data' is not defined

Tenga en cuenta que para los índices enteros, el indexador ``ix`` está sujeto a las mismas fuentes potenciales de confusión que se han discutido para los objetos ``Series`` indexados por enteros.

Dentro de estos indexadores se puede utilizar cualquiera de los patrones de acceso a datos conocidos del estilo NumPy.
Por ejemplo, en el indexador ``loc`` podemos combinar enmascaramiento e indexación de fantasía como en lo siguiente:

In [111]:
data.loc[data['density']> 100, ['pop', 'density']]

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; esto se hace de la manera estándar a la que puede estar acostumbrado de trabajar con NumPy:

In [112]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Para mejorar tu fluidez en la manipulación de datos en Pandas, te sugiero que pases algún tiempo con un simple ``DataFrame`` y explores los tipos de indexación, troceado, enmascaramiento e indexación de fantasía que permiten estos diversos enfoques de indexación.

### Convenciones de indexación adicionales

Hay un par de convenciones de indexación adicionales que pueden parecer contradictorias con la discusión anterior, pero que sin embargo pueden ser muy útiles en la práctica.
**En primer lugar, mientras que *indexar* se refiere a columnas, *rebanar* se refiere a filas:**.

In [ ]:
data['Florida':'Illinois']

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [ ]:
data.loc[:, 'area':'density']

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Estas rebanadas también pueden referirse a filas por número en lugar de por índice:

In [ ]:
data[1:3]

,area,pop,density
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


Del mismo modo, las operaciones de enmascaramiento directo también se interpretan por filas en lugar de por columnas:

In [ ]:
data[data['density'] > 100]

,area,pop,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


Estas dos convenciones son sintácticamente similares a las de un array de NumPy, y aunque no se ajusten exactamente al molde de las convenciones de Pandas, son bastante útiles en la práctica.